In [2]:
%%capture
#  libraries
!pip install --upgrade openai
!pip install --upgrade langchain
# store the API key in a .env file
!pip install --upgrade python-dotenv

In [40]:
import openai
#from langchain.llms import OpenAI
from dotenv import load_dotenv

import os
import IPython

Load environment variables. You can use anything you like but I used python-dotenv. Just create a .env file with your OPENAI_API_KEY then load it.

In [41]:
load_dotenv()

# API configuration
openai.api_key = os.getenv("OPENAI_API_KEY")

# for LangChain
#os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

In [42]:
def set_open_params(
    model="gpt-3.5-turbo",
    temperature=0.7,
    max_tokens=256,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0,
):
    """ set openai parameters"""

    openai_params = {}    

    openai_params['model'] = model
    openai_params['temperature'] = temperature
    openai_params['max_tokens'] = max_tokens
    openai_params['top_p'] = top_p
    openai_params['frequency_penalty'] = frequency_penalty
    openai_params['presence_penalty'] = presence_penalty
    return openai_params

def get_completion(params, messages):
    """ GET completion from openai api"""

    response = openai.chat.completions.create(
        model = params['model'],
        messages = messages,
        temperature = params['temperature'],
        max_tokens = params['max_tokens'],
        top_p = params['top_p'],
        frequency_penalty = params['frequency_penalty'],
        presence_penalty = params['presence_penalty'],
    )
    return response

## Basic prompt example:  Convert to Hindi 

In [43]:
# basic example
params = set_open_params()

prompt = "Translate the following English sentence to Hindi: 'The quick brown fox jumps over the lazy dog.'"

messages = [
    {
        "role": "user",
        "content": prompt
    }
]

response = get_completion(params, messages)
print(response.choices[0].message.content)

'तेज़ भूरा लोमड़ी सुस्त कुत्ते पर छलांग लगाता है।'


In [19]:
params = set_open_params(temperature=0)
response = get_completion(params, messages)
IPython.display.Markdown(response.choices[0].message.content)

 blue and dotted with fluffy white clouds.

## Text Summarization


In [44]:
params = set_open_params(temperature=0, max_tokens=100)
prompt = """Neural networks are computational models inspired by the structure and function of the human brain. They are a fundamental component of modern artificial intelligence, enabling machines to learn from data and perform tasks that traditionally required human intelligence.

At their core, neural networks consist of interconnected nodes, or "neurons," organized into layers. Each neuron processes input data and passes it through an activation function to produce an output. Through a process called training, neural networks adjust the strength of connections between neurons to optimize their performance on a specific task.

One of the key strengths of neural networks is their ability to automatically discover patterns and relationships in complex data, making them well-suited for tasks such as image and speech recognition, natural language processing, and predictive analytics.

As technology advances, neural networks continue to evolve, with architectures such as convolutional neural networks (CNNs) for image processing, recurrent neural networks (RNNs) for sequential data, and transformers for natural language understanding pushing the boundaries of AI capabilities.
Summarize succintly, one liner like I am 5:"""

messages = [
    {
        "role": "user",
        "content": prompt
    }
]

response = get_completion(params, messages)
IPython.display.Markdown(response.choices[0].message.content)

Neural networks are like computer brains that can learn from data and do smart things.

## Question Answering

In [45]:
prompt = """Answer the question based on the context below. Keep the answer short and concise. Respond "Unsure about answer" if not sure about the answer.

Context: If Earth is moving so fast, why don’t we fly off? Thank gravity for that. Gravity is the force of attraction between all objects in the universe. An object’s gravity depends on its mass —its total amount of matter , or “stuff.” The more massive the object, the stronger the gravitational force. And the closer two objects are, the stronger the gravitational pull between them. Gravity is what keeps your feet on the ground—and what keeps Earth and the planets orbiting around the Sun instead of floating away. 

When you jump, you always fall back to Earth. What goes up must come down, right? Not quite! Something can go up and not come down if it reaches the escape velocity, the speed at which it breaks free from a planet’s gravitational pull. That’s how rockets work. Their engines are designed to push the rocket up so hard that it moves fast enough to get away. The universe is full of “runaway” stars and planets that escaped the gravity of their neighbors..

Question: What is gravity?

Answer:"""

messages = [
    {
        "role": "user",
        "content": prompt
    }
]

response = get_completion(params, messages)
IPython.display.Markdown(response.choices[0].message.content)

Gravity is the force of attraction between all objects in the universe.

## Sentiment analysis  (lets check some sarcasm) 

### Zero-Shot Prompting


In [47]:
prompt = """Classify the text into neutral, negative or positive.

Text: I walked into my hotel room and wondered if the interior decorators thought orange was the new black..

Sentiment:"""

messages = [
    {
        "role": "user",
        "content": prompt
    }
]

response = get_completion(params, messages)
IPython.display.Markdown(response.choices[0].message.content)

Negative

## SQL code  Generation


In [55]:
params = set_open_params(
    temperature=0,
    max_tokens=500,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0,
)
prompt = "\"\"\"\nTable departments, columns = [DepartmentId, DepartmentName]\nTable students, columns = [DepartmentId, StudentId, StudentName, JobSalary]\nCreate a MySQL query for all students in the Computer Science Department with fourth highest jobsalary without using LIMIT  OFFSET\n\"\"\""

messages = [
    {
        "role": "user",
        "content": prompt
    }
]

response = get_completion(params, messages)
IPython.display.Markdown(response.choices[0].message.content)

SELECT s.StudentName, s.JobSalary
FROM students s
WHERE s.DepartmentId = (SELECT d.DepartmentId
                        FROM departments d
                        WHERE d.DepartmentName = 'Computer Science')
AND s.JobSalary = (SELECT DISTINCT s2.JobSalary
                   FROM students s2
                   WHERE s2.DepartmentId = (SELECT d2.DepartmentId
                                            FROM departments d2
                                            WHERE d2.DepartmentName = 'Computer Science')
                   ORDER BY s2.JobSalary DESC
                   LIMIT 1 OFFSET 3)

## python code  Generation


In [58]:
prompt = """Write a python Problem Statement:

You are tasked with developing a Python program to analyze the sales data of a retail store. The program should read a CSV file containing sales data, including the product ID, product name, quantity sold, and sales amount. Upon reading the data, the program should perform the following tasks:

Calculate the total sales amount.
Determine the best-selling product (the product with the highest quantity sold).
Identify the total sales for each product category.
Generate a summary report containing the total sales amount, the best-selling product, and the total sales for each product category.
The program should provide a user-friendly interface to display the summary report and allow the user to export the report to a CSV file for further analysis.

Example Input:

CSV file containing sales data (product ID, product name, quantity sold, sales amount)
Example Output:

Total sales amount: $XXXX
Best-selling product: ProductName (Quantity Sold: XX)
Total sales for each product category:
Category1: $XXXX
Category2: $XXXX
...
Summary report exported to CSV file: summary_report.csv
Additional Requirements:

The program should handle cases where the input file is missing or incorrectly formatted.
It should be able to handle large datasets efficiently.
Error messages and warnings should be displayed to guide the user in case of issues with the input data or during the execution of the program.
Develop a Python program that fulfills the requirements outlined above."""

messages = [
    {
        "role": "user",
        "content": prompt
    }
]
params = set_open_params(
    temperature=0,
    max_tokens=4000,
    top_p=1
)
response = get_completion(params, messages)
IPython.display.Markdown(response.choices[0].message.content)

```python
import csv

def read_sales_data(file_path):
    sales_data = []
    try:
        with open(file_path, 'r') as file:
            reader = csv.reader(file)
            next(reader)  # Skip header row
            for row in reader:
                sales_data.append({
                    'product_id': row[0],
                    'product_name': row[1],
                    'quantity_sold': int(row[2]),
                    'sales_amount': float(row[3])
                })
    except FileNotFoundError:
        print("File not found. Please provide a valid file path.")
    except IndexError:
        print("Incorrect file format. Please make sure the file contains product ID, product name, quantity sold, and sales amount.")
    
    return sales_data

def calculate_total_sales(sales_data):
    total_sales = sum([sale['sales_amount'] for sale in sales_data])
    return total_sales

def find_best_selling_product(sales_data):
    best_selling_product = max(sales_data, key=lambda x: x['quantity_sold'])
    return best_selling_product

def calculate_total_sales_per_category(sales_data):
    total_sales_per_category = {}
    for sale in sales_data:
        category = sale['product_name'].split()[0]  # Assuming the category is the first word in the product name
        if category in total_sales_per_category:
            total_sales_per_category[category] += sale['sales_amount']
        else:
            total_sales_per_category[category] = sale['sales_amount']
    return total_sales_per_category

def generate_summary_report(total_sales, best_selling_product, total_sales_per_category):
    print(f"Total sales amount: ${total_sales}")
    print(f"Best-selling product: {best_selling_product['product_name']} (Quantity Sold: {best_selling_product['quantity_sold']})")
    print("Total sales for each product category:")
    for category, total_sales in total_sales_per_category.items():
        print(f"{category}: ${total_sales}")
    
    with open('summary_report.csv', 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["Category", "Total Sales"])
        for category, total_sales in total_sales_per_category.items():
            writer.writerow([category, total_sales])
    
if __name__ == "__main__":
    file_path = 'sales_data.csv'
    sales_data = read_sales_data(file_path)
    
    if sales_data:
        total_sales = calculate_total_sales(sales_data)
        best_selling_product = find_best_selling_product(sales_data)
        total_sales_per_category = calculate_total_sales_per_category(sales_data)
        
        generate_summary_report(total_sales, best_selling_product, total_sales_per_category)
```
```python
# Sample sales_data.csv file content:
# product_id,product_name,quantity_sold,sales_amount
# 1,ProductA 123,10,100.0
# 2,ProductB 456,20,200.0
# 3,ProductC 789,30,300.0
```

## Reasoning

In [59]:
prompt = """The odd numbers in this group add up to an even number: 15, 32, 5, 13, 82, 7, 1. 

Solve by breaking the problem into steps. First, identify the odd numbers, add them, and indicate whether the result is odd or even."""

messages = [
    {
        "role": "user",
        "content": prompt
    }
]

response = get_completion(params, messages)
IPython.display.Markdown(response.choices[0].message.content)

Odd numbers in the group: 15, 5, 13, 7, 1

Adding the odd numbers: 15 + 5 + 13 + 7 + 1 = 41

The sum of the odd numbers (41) is an odd number.

 ## Few-shot prompts  
 ### 1-shot | 3-shot | 5-shot --- number of exmaples

In [60]:
prompt = """The odd numbers in this group add up to an even number: 4, 8, 9, 15, 12, 2, 1.
A: The answer is False.

The odd numbers in this group add up to an even number: 17,  10, 19, 4, 8, 12, 24.
A: The answer is True.

The odd numbers in this group add up to an even number: 16,  11, 14, 4, 8, 13, 24.
A: The answer is True.

The odd numbers in this group add up to an even number: 17,  9, 10, 12, 13, 4, 2.
A: The answer is False.

The odd numbers in this group add up to an even number: 15, 32, 5, 13, 82, 7, 1. 
A:"""

messages = [
    {
        "role": "user",
        "content": prompt
    }
]

response = get_completion(params, messages)
IPython.display.Markdown(response.choices[0].message.content)

The answer is True.


## Chain of thoughts 
Add step by steps reasoning about solving in the example . 

In [61]:
prompt = """The odd numbers in this group add up to an even number: 4, 8, 9, 15, 12, 2, 1.
A: Adding all the odd numbers (9, 15, 1) gives 25. The answer is False.

The odd numbers in this group add up to an even number: 15, 32, 5, 13, 82, 7, 1. 
A:"""

messages = [
    {
        "role": "user",
        "content": prompt
    }
]

response = get_completion(params, messages)
IPython.display.Markdown(response.choices[0].message.content)

KeyboardInterrupt: 

## Zero-shot CoT

All we need to do add "Let's think step by step."

In [62]:
prompt = """I went to the market and bought 10 apples. I gave 2 apples to the neighbor and 2 to the repairman. I then went and bought 5 more apples and ate 1. How many apples did I remain with?

Let's think step by step."""

messages = [
    {
        "role": "user",
        "content": prompt
    }
]

response = get_completion(params, messages)
IPython.display.Markdown(response.choices[0].message.content)

Initially, you had 10 apples.
After giving 2 apples to the neighbor, you had 10 - 2 = 8 apples.
After giving 2 apples to the repairman, you had 8 - 2 = 6 apples.
After buying 5 more apples, you had 6 + 5 = 11 apples.
After eating 1 apple, you remained with 11 - 1 = 10 apples.

So, you remained with 10 apples in the end.